# General code for creating and saving the CAI data

In [ ]:
#importing necessary libraries
import os                   #to create new directories
import shutil               #to delete directories
import pandas as pd         #to convert excel data into dataframes into lists
from Bio import SeqIO       #to read/write the fasta files
from xlwt import Workbook   #to save excel files
from matplotlib import pyplot as plt    #to plot graphs
import numpy as np                      #to deal with arrays

#getting list of all strains from folder
list_strains= os.listdir(r"C:\Users\husay\Desktop\Biotech\Species_genomes\Ecoli_strains")

# read by default 1st sheet of an excel file
dataframe1 = pd.read_excel(r"C:\Users\husay\Desktop\Biotech\adaptation_index.xlsx")
# path to cai data excel sheet
codon_list=dataframe1.values.tolist()#convert pandas database to list

data_frame = pd.read_excel(r"C:\Users\husay\Desktop\Biotech\New_final_list.xlsx")

gene_list=data_frame.values.tolist()

from openpyxl import load_workbook
workbook = load_workbook(r"C:\Users\husay\Desktop\Biotech\New_final_list.xlsx")
sheet=workbook.active

#function to search item in seq_record irrespective of case
def search(p,t):
   for classes in t:
       if p.lower()==classes.lower():
           return True
   return False

for gene in gene_list:
    gene_index=gene_list.index(gene)
    if gene_index<1743: continue
    gene_name=gene[1]
    protein_name=gene[9]
    protein_type=gene[8]
    count=0
    print(gene_name, protein_name)
    #looping the strains
    for strain in list_strains:
        gene=0;#to check whether gene is even present or not
    
        #below file paths can be changed
        file_in =r'C:\Users\husay\Desktop\Biotech\Species_genomes\Ecoli_strains/'+ strain +'\cds_from_genomic.fna'
        #remove folders without cds file
        if not os.path.exists(file_in):
            shutil.rmtree(r'C:\Users\husay\Desktop\Biotech\Species_genomes\Ecoli_strains/'+ strain, ignore_errors=True)
            continue
    
        filepath = os.path.join(r'C:\Users\husay\Desktop\Biotech\Species_genomes\Ecoli_strains/'+strain+'/'+gene_name+' ('+protein_type+')', gene_name+'.fna')
        
      #if gene present make fasta file ie f_out
        for seq_record in SeqIO.parse(open(file_in,'r'), 'fasta'):
            g=search('[gene='+gene_name+']',seq_record.description.split())
            p=search('protein='+protein_name+'] ',seq_record.description.split('['))
            #print(seq_record.description.split('['))
            if g or p :
                #print(g,p)
                if not os.path.exists(r'C:\Users\husay\Desktop\Biotech\Species_genomes\Ecoli_strains/'+strain+'/'+gene_name+' ('+protein_type+')'):
                    os.makedirs(r'C:\Users\husay\Desktop\Biotech\Species_genomes\Ecoli_strains/'+strain+'/'+gene_name+' ('+protein_type+')')
                with open(filepath, 'w') as f_out:
                    r=SeqIO.write(seq_record, f_out, 'fasta')
                    if r!=1: print('Error while writing sequence:  ' + seq_record.id)
                    gene=1;break;
                    
        if gene==0:
            #print("Gene "+gene_name+" not found in strain "+strain+"\n\n\n\n\n")
            continue;
    
        data=open(r'C:\Users\husay\Desktop\Biotech\Species_genomes\Ecoli_strains/'+strain+'/'+gene_name+' ('+protein_type+')'+'/'+gene_name+'.fna')
        raw_seq= data.read() #open the fasta file and read the sequence
        data.close()
    
        seq=raw_seq[raw_seq.find('ATG'):] #remove the first line of meta data
        seq=seq.replace("\n","")#replace unnecessary characters
    
        codons=[] #gene seq partitioned into codons
        for i in range(0,len(seq),3):
            string=seq[i:i+3:1]
            codons.append(string)
    
        gene_length=len(codons)
    
        # Workbook is created
        wb = Workbook()
  
        # add_sheet is used to create sheet.
        sheet1 = wb.add_sheet('CAI values')
        sheet1.write(0, 0, 'Strain')
        sheet1.write(1, 0, strain)
        sheet1.write(0, 1, 'CAI values')
        sheet1.write(0, 2, 'mwf=15')
        sheet1.write(0, 3, 'mwf=20')
        sheet1.write(0, 4, 'mwf=25')
    
        cai=[] #cai value list
        for i in codons:
            for j in codon_list: #from excel sheet
                if i==j[0] :
                    cai.append(j[1])
        for i in range(len(cai)):  
            sheet1.write(i+1, 1, cai[i])
            
        count+=1   
        for n in range (15,26,5):
            #implementing moving window frame
            mwf=[] #list containing moving window frame data
            for i in range (n,len(cai)):
                pdt=1
                for j in range (n):
                    pdt*=cai[i-j]       
                mwf.append(pdt)
        
            for i in range(len(mwf)):
                sheet1.write(i+1, int(n/5)-1, mwf[i])
                
        #saving values in excel sheet            
        #wb.save('CAI Values ('+gene_name+') .xls')
        if not os.path.exists(r"C:\Users\husay\Desktop\Biotech\Species_genomes\Ecoli_strains/"+strain+"/"+gene_name+' ('+protein_type+')'+"/"):
            os.makedirs(r"C:\Users\husay\Desktop\Biotech\Species_genomes\Ecoli_strains/"+strain+"/"+gene_name+' ('+protein_type+')'+"/")
        wb.save(r"C:\Users\husay\Desktop\Biotech\Species_genomes\Ecoli_strains/"+strain+"/"+gene_name+' ('+protein_type+')'+"/CAI Values.xls")

    sheet["L"+str(gene_index+2)]=count
    if gene_index%100==0:
        workbook.save(filename=r"C:\Users\husay\Desktop\Biotech\New_final_list.xlsx")
    #Count.append(count)

In [ ]:
import os
import pandas as pd
list_strains= os.listdir(r"C:\Users\husay\Desktop\Biotech\Species_genomes\Ecoli_strains")
data_frame = pd.read_excel(r"C:\Users\husay\Desktop\Biotech\New_final_list.xlsx")

gene_list=data_frame.values.tolist()

from openpyxl import load_workbook
workbook = load_workbook(r"C:\Users\husay\Desktop\Biotech\New_final_list.xlsx")
sheet=workbook.active

for gene in gene_list:
    gene_index=gene_list.index(gene)
    
    file_name=gene[1]+' ('+gene[8]+')'
    count=0
    #print(gene_index)
    #looping the strains
    for strain in list_strains:
        name_list=os.listdir(r"C:\Users\husay\Desktop\Biotech\Species_genomes\Ecoli_strains/"+strain)
        if file_name in name_list:
            count+=1
    sheet["L"+str(gene_index+2)]=count
    if gene_index%100==0:
        workbook.save(filename=r"C:\Users\husay\Desktop\Biotech\New_final_list.xlsx")

In [5]:
workbook.save(filename=r"C:\Users\husay\Desktop\Biotech\New_final_list.xlsx")
#print(len(Count))

In [4]:
from openpyxl import load_workbook
workbook = load_workbook(r"C:\Users\husay\Desktop\Biotech\New_final_list.xlsx")
sheet=workbook.active
for i in range(len(Count)):
    sheet["L"+str(404+i)]=Count[i]
workbook.save(filename=r"C:\Users\husay\Desktop\Biotech\New_final_list.xlsx")

Reading from excel sheet

In [3]:
dataframe = pd.read_excel(r"C:\Users\husay\Desktop\Biotech\List of Genes (E.coli K-12 substr. MG1655).xlsx")
print(dataframe["Gene Name"])

0       rfbC
1       yehE
2        tmk
3       yadG
4       sthA
        ... 
4759    fdoH
4760    yfiM
4761    guaB
4762    uxuA
4763    ariR
Name: Gene Name, Length: 4764, dtype: object


In [6]:
gene_list=dataframe.values.tolist()
print(gene_list[1][0])
#first index is row, second index is column

yehE
